# Demo of graph injection attack

In this demo, we will show a completed process of applying the graph injection attack in a [refined CORA](https://github.com/THUDM/Refined-cora-citeseer) dataset.

In [79]:
import random
import pickle
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Data preparation

### 1.1. Load CORA (refined) dataset

In [2]:
with open("./data/Refined-cora-citeseer/corax_adj.pkl", 'rb') as f:
    raw_adj = pickle.load(f)
with open("./data/Refined-cora-citeseer/corax_features.pkl", 'rb') as f:
    raw_features = pickle.load(f)
with open("./data/Refined-cora-citeseer/corax_labels.pkl", 'rb') as f:
    raw_labels = pickle.load(f)
    raw_labels = np.argmax(raw_labels, axis=1)

In [3]:
num_nodes = raw_features.shape[0]
num_edges = raw_adj.getnnz() // 2
num_features = raw_features.shape[1]
num_classes = raw_labels.max() + 1

In [4]:
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[range(1180)] = True
val_mask[range(1180, 2180)] = True
test_mask[range(2180, 2680)] = True
num_train = int(torch.sum(train_mask))
num_val = int(torch.sum(val_mask))
num_test = int(torch.sum(test_mask))

In [5]:
print("Number of nodes: {}.".format(num_nodes))
print("Number of edges: {}.".format(num_edges))
print("Number of features: {}.".format(num_features))
print("Number of classes: {}.".format(num_classes))
print("Number of train samples: {}.".format(num_train))
print("Number of val samples: {}.".format(num_val))
print("Number of test samples: {}.".format(num_test))
print("Feature range: [{:.4f}, {:.4f}]".format(raw_features.min(), raw_features.max()))

Number of nodes: 2680.
Number of edges: 5148.
Number of features: 302.
Number of classes: 7.
Number of train samples: 1180.
Number of val samples: 1000.
Number of test samples: 500.
Feature range: [-2.2968, 2.4000]


### 1.2. Preprocessing

In [6]:
def eval_acc(pred, labels, mask=None):
    if mask is None:
        return (torch.argmax(pred, dim=1) == labels).float().sum() / len(pred)
    else:
        return (torch.argmax(pred[mask], dim=1) == labels[mask]).float().sum() / int(torch.sum(mask))

In [74]:
def eval_model(model, features, adj, mask=None):
    model.eval()
    pred = model(features, adj, dropout=0)
    pred_label = torch.argmax(pred, dim=1)
    acc = eval_acc(pred[:len(mask)], labels, mask=mask)

    return acc

In [7]:
def adj_to_tensor(adj):
    sparse_row = torch.LongTensor(adj.row).unsqueeze(1)
    sparse_col = torch.LongTensor(adj.col).unsqueeze(1)
    sparse_concat = torch.cat((sparse_row, sparse_col), 1)
    sparse_data = torch.FloatTensor(adj.data)
    adj_tensor = torch.sparse.FloatTensor(sparse_concat.t(), sparse_data, torch.Size(adj.shape))

    return adj_tensor

In [8]:
def adj_norm(adj, order=-0.5):
    adj = sp.eye(adj.shape[0]) + adj
    for i in range(len(adj.data)):
        if adj.data[i] > 0 and adj.data[i] != 1:
            adj.data[i] = 1
    adj = sp.coo_matrix(adj)

    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, order).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    adj = d_mat_inv_sqrt @ adj @ d_mat_inv_sqrt

    return adj.tocoo()

In [9]:
device = 'cpu'  #'cuda:0'

adj = raw_adj
adj = adj_norm(adj)
adj = adj_to_tensor(adj).to(device)

features = torch.FloatTensor(raw_features)
labels = torch.LongTensor(raw_labels)

## 2. Model preparation

### 2.1. Example of GCN ([Graph Convolutional Network](https://arxiv.org/abs/1609.02907))

In [10]:
class GCNConv(nn.Module):
    def __init__(self, in_features, out_features, activation=None, dropout=False):
        super(GCNConv, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.linear = nn.Linear(in_features, out_features)
        self.activation = activation
        self.dropout = dropout
        self.reset_parameters()

    def reset_parameters(self):
        if self.activation == F.leaky_relu:
            gain = nn.init.calculate_gain('leaky_relu')
        else:
            gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.linear.weight, gain=gain)

    def forward(self, x, adj, dropout=0):
        x = self.linear(x)
        x = torch.spmm(adj, x)
        if self.activation is not None:
            x = self.activation(x)
        if self.dropout:
            x = F.dropout(x, dropout)

        return x


class GCN(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, activation=F.relu, dropout=True):
        super(GCN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        if type(hidden_features) is int:
            hidden_features = [hidden_features]

        self.layers = nn.ModuleList()
        self.layers.append(GCNConv(in_features, hidden_features[0], activation=activation, dropout=dropout))
        for i in range(len(hidden_features) - 1):
            self.layers.append(
                GCNConv(hidden_features[i], hidden_features[i + 1], activation=activation, dropout=dropout))
        self.layers.append(GCNConv(hidden_features[-1], out_features))
        self.reset_parameters()

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, x, adj, dropout=0):
        for layer in self.layers:
            x = layer(x, adj, dropout=dropout)

        return x

In [11]:
model = GCN(in_features=num_features, 
            out_features=num_classes, 
            hidden_features=[16, 16], 
            activation=F.relu)
model.to(device)
print(model)

GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=302, out_features=16, bias=True)
    )
    (1): GCNConv(
      (linear): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (linear): Linear(in_features=16, out_features=7, bias=True)
    )
  )
)


### 2.2. Model training

In [12]:
n_epoch = 200
eval_every = 10
dropout = 0.5

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [13]:
model.train()

for epoch in range(n_epoch):
    logits = model(features, adj, dropout)
    logp = F.log_softmax(logits, 1)
    train_loss = F.nll_loss(logp[train_mask], labels[train_mask])
    val_loss = F.nll_loss(logp[val_mask], labels[val_mask])

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch % eval_every == 0:
        train_acc = eval_acc(logp, labels, train_mask)
        val_acc = eval_acc(logp, labels, val_mask)
        print('Epoch {:05d} | Train Loss {:.4f} | Train Acc {:.4f} | Val Loss {:.4f} | Val Acc {:.4f}'.format(
                    epoch, train_loss, train_acc, val_loss, val_acc))

Epoch 00000 | Train Loss 2.0604 | Train Acc 0.1195 | Val Loss 2.0613 | Val Acc 0.1340
Epoch 00010 | Train Loss 1.4966 | Train Acc 0.4907 | Val Loss 1.5530 | Val Acc 0.4660
Epoch 00020 | Train Loss 1.1267 | Train Acc 0.6144 | Val Loss 1.1453 | Val Acc 0.6090
Epoch 00030 | Train Loss 0.8761 | Train Acc 0.7381 | Val Loss 0.9028 | Val Acc 0.7050
Epoch 00040 | Train Loss 0.7892 | Train Acc 0.7500 | Val Loss 0.8046 | Val Acc 0.7270
Epoch 00050 | Train Loss 0.7288 | Train Acc 0.7602 | Val Loss 0.7490 | Val Acc 0.7590
Epoch 00060 | Train Loss 0.6538 | Train Acc 0.7712 | Val Loss 0.6848 | Val Acc 0.7670
Epoch 00070 | Train Loss 0.6248 | Train Acc 0.8017 | Val Loss 0.6797 | Val Acc 0.7790
Epoch 00080 | Train Loss 0.5734 | Train Acc 0.8093 | Val Loss 0.6743 | Val Acc 0.7860
Epoch 00090 | Train Loss 0.5276 | Train Acc 0.8364 | Val Loss 0.6462 | Val Acc 0.7940
Epoch 00100 | Train Loss 0.4916 | Train Acc 0.8475 | Val Loss 0.5938 | Val Acc 0.8110
Epoch 00110 | Train Loss 0.4839 | Train Acc 0.8415 | V

In [14]:
torch.save(model.state_dict(), "./saved_models/model_gcn.pt")

### 2.3. Model inference

In [76]:
acc = eval_model(model, features, adj, test_mask)
print("Test accuracy: {:.4f}".format(acc))

Test accuracy: 0.8660


## 3. Example of graph injection attack (based on [FGSM](https://arxiv.org/abs/1412.6572))

### 3.1. Generate connections of injected nodes (randomly)

In [16]:
num_inject = 10
num_edge_max = 50

In [32]:
def injection(adj, n_inject, n_node, n_edge_max, test_index):
    n_test = test_index.shape[0]
    new_edges_x = []
    new_edges_y = []
    new_data = []
    for i in range(n_inject):
        islinked = np.zeros(n_test)
        for j in range(n_edge_max):
            x = i + n_node

            yy = random.randint(0, n_test - 1)
            while islinked[yy] > 0:
                yy = random.randint(0, n_test - 1)

            y = test_index[yy]
            new_edges_x.extend([x, y])
            new_edges_y.extend([y, x])
            new_data.extend([1, 1])

    add1 = sp.csr_matrix((n_inject, n_node))
    add2 = sp.csr_matrix((n_node + n_inject, n_inject))
    adj_attack = sp.vstack([adj, add1])
    adj_attack = sp.hstack([adj_attack, add2])
    adj_attack.row = np.hstack([adj_attack.row, new_edges_x])
    adj_attack.col = np.hstack([adj_attack.col, new_edges_y])
    adj_attack.data = np.hstack([adj_attack.data, new_data])
    
    return adj_attack

In [33]:
adj_attack = injection(adj=raw_adj,
                       n_inject=num_inject,
                       n_node=num_nodes,
                       n_edge_max=num_edge_max,
                       test_index=torch.where(test_mask == True)[0])

In [34]:
adj_attack = adj_norm(adj_attack)
adj_attack = adj_to_tensor(adj_attack)
adj_attack

tensor(indices=tensor([[   0,    0,    0,  ..., 2689, 2689, 2689],
                       [   0,  632, 1848,  ..., 2652, 2676, 2689]]),
       values=tensor([0.2500, 0.2500, 0.2236,  ..., 0.0632, 0.0707, 0.0200]),
       size=(2690, 2690), nnz=13938, layout=torch.sparse_coo)

### 3.2. Update features by FGSM (Fast Gradient Sign Method)

In [35]:
def fgsm(features, features_attack, adj_attack, labels, test_mask, n_epoch, epsilon, feat_min, feat_max, device='cpu'):
    n_total = features.shape[0]
    for i in range(n_epoch):
        features_attack.requires_grad_(True)
        features_attack.retain_grad()
        
        features_concat = torch.cat((features, features_attack), dim=0)
        pred = model(features_concat, adj_attack) 
        pred_loss = -F.nll_loss(pred[:n_total][test_mask], labels[test_mask]).to(device)
        
        model.zero_grad()
        pred_loss.backward()
        grad = features_attack.grad.data
        features_attack = features_attack.clone() + epsilon * grad.sign()

        features_attack = torch.clamp(features_attack, feat_min, feat_max)
        features_attack = features_attack.detach()
            
        print("Epoch {}, Loss: {:.5f}, Test acc: {:.5f}".format(i, pred_loss,
                                                                eval_acc(pred[:n_total][test_mask],
                                                                         labels[test_mask])))
        
    return features_attack

In [36]:
features_attack = torch.zeros([num_inject, num_features], dtype=torch.float).to(device)
features_attack.requires_grad_(True)
features_attack = fgsm(features=features, 
                       features_attack=features_attack,
                       adj_attack=adj_attack,
                       labels=labels,
                       test_mask=test_mask,
                       n_epoch=50,
                       epsilon=0.1,
                       feat_min=-2.0,
                       feat_max=2.0,
                       device=device)

Epoch 0, Loss: 5.20593, Test acc: 0.85600
Epoch 1, Loss: 5.62591, Test acc: 0.80600
Epoch 2, Loss: 6.02167, Test acc: 0.75000
Epoch 3, Loss: 6.41083, Test acc: 0.71400
Epoch 4, Loss: 6.80009, Test acc: 0.68800
Epoch 5, Loss: 7.18971, Test acc: 0.67600
Epoch 6, Loss: 7.58298, Test acc: 0.65400
Epoch 7, Loss: 7.97815, Test acc: 0.64400
Epoch 8, Loss: 8.37593, Test acc: 0.63600
Epoch 9, Loss: 8.77523, Test acc: 0.63200
Epoch 10, Loss: 9.17685, Test acc: 0.61800
Epoch 11, Loss: 9.58056, Test acc: 0.61400
Epoch 12, Loss: 9.98615, Test acc: 0.60400
Epoch 13, Loss: 10.39431, Test acc: 0.59200
Epoch 14, Loss: 10.80378, Test acc: 0.58200
Epoch 15, Loss: 11.21489, Test acc: 0.57600
Epoch 16, Loss: 11.62671, Test acc: 0.57400
Epoch 17, Loss: 12.03965, Test acc: 0.56600
Epoch 18, Loss: 12.45364, Test acc: 0.56400
Epoch 19, Loss: 12.86957, Test acc: 0.55600
Epoch 20, Loss: 13.28626, Test acc: 0.55600
Epoch 21, Loss: 13.30542, Test acc: 0.55600
Epoch 22, Loss: 13.32370, Test acc: 0.55600
Epoch 23, L

In [57]:
# Save results
np.save("./results/features_attack.npy", features_attack.detach().numpy())
with open("./results/adj_attack.pkl", 'wb') as f:
    pickle.dump(adj_attack, f)

## 4. Evaluation on other GNN models

### 4.1. Example of GIN ([Graph Isomorphism Network](https://arxiv.org/abs/1810.00826))

In [41]:
class GINConv(nn.Module):
    def __init__(self, in_features, out_features, activation=F.relu, eps=0, batchnorm=False, dropout=False):
        super(GINConv, self).__init__()
        self.linear1 = nn.Linear(in_features, out_features)
        self.linear2 = nn.Linear(out_features, out_features)
        self.activation = activation
        self.eps = torch.nn.Parameter(torch.Tensor([eps]))
        self.batchnorm = batchnorm
        if batchnorm:
            self.norm = nn.BatchNorm1d(out_features)
        self.dropout = dropout

    def reset_parameters(self):
        if self.activation == F.leaky_relu:
            gain = nn.init.calculate_gain('leaky_relu')
        else:
            gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.linear.weight, gain=gain)

    def forward(self, x, adj, dropout=0):
        y = torch.spmm(adj, x)
        x = y + (1 + self.eps) * x
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        if self.batchnorm:
            x = self.norm(x)
        if self.dropout:
            x = F.dropout(x, dropout)

        return x


class GIN(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, activation=F.relu, dropout=True):
        super(GIN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        if type(hidden_features) is int:
            hidden_features = [hidden_features]
        self.layers = nn.ModuleList()

        self.layers.append(GINConv(in_features, hidden_features[0], activation=activation, dropout=dropout))
        for i in range(len(hidden_features) - 1):
            self.layers.append(
                GINConv(hidden_features[i], hidden_features[i + 1], activation=activation))
        self.linear1 = nn.Linear(hidden_features[-2], hidden_features[-1])
        self.linear2 = nn.Linear(hidden_features[-1], out_features)

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, x, adj, dropout=0):
        for layer in self.layers:
            x = layer(x, adj, dropout=dropout)
        x = F.relu(self.linear1(x))
        x = F.dropout(x, dropout)
        x = self.linear2(x)

        return x

In [60]:
model_gin = GIN(in_features=num_features, 
            out_features=num_classes, 
            hidden_features=[16, 16], 
            activation=F.relu)
model_gin.load_state_dict(torch.load("./saved_models/model_gin.pt"))
model_gin.to(device)
print(model_gin)

GIN(
  (layers): ModuleList(
    (0): GINConv(
      (linear1): Linear(in_features=302, out_features=16, bias=True)
      (linear2): Linear(in_features=16, out_features=16, bias=True)
    )
    (1): GINConv(
      (linear1): Linear(in_features=16, out_features=16, bias=True)
      (linear2): Linear(in_features=16, out_features=16, bias=True)
    )
  )
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)


In [80]:
acc = eval_model(model_gin, features, adj, test_mask)
print("Test accuracy (Original): {:.4f}".format(acc))

Test accuracy (Original): 0.7980


In [81]:
acc = eval_model(model_gin, torch.cat([features, features_attack]), adj_attack, test_mask)
print("Test accuracy (Attacked): {:.4f}".format(acc))

Test accuracy (Attacked): 0.4960


### 4.2. Example of TAGCN ([Topological Adaptive Graph Convolutional Network](https://arxiv.org/abs/1710.10370))

In [61]:
class TAGConv(nn.Module):
    def __init__(self, in_features, out_features, k=2, activation=None, dropout=False, batchnorm=False):
        super(TAGConv, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.linear = nn.Linear(in_features * (k + 1), out_features)
        self.batchnorm = batchnorm
        if batchnorm:
            self.norm_func = nn.BatchNorm1d(out_features, affine=False)
        self.activation = activation
        self.dropout = dropout
        self.k = k
        self.reset_parameters()

    def reset_parameters(self):
        if self.activation == F.leaky_relu:
            gain = nn.init.calculate_gain('leaky_relu')
        else:
            gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.linear.weight, gain=gain)

    def forward(self, x, adj, dropout=0):

        fstack = [x]
        for i in range(self.k):
            y = torch.spmm(adj, fstack[-1])
            fstack.append(y)
        x = torch.cat(fstack, dim=-1)
        x = self.linear(x)
        if self.batchnorm:
            x = self.norm_func(x)
        if not (self.activation is None):
            x = self.activation(x)
        if self.dropout:
            x = F.dropout(x, dropout)

        return x


class TAGCN(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, k, activation=F.leaky_relu, dropout=True):
        super(TAGCN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        if type(hidden_features) is int:
            hidden_features = [hidden_features]

        self.layers = nn.ModuleList()
        self.layers.append(TAGConv(in_features, hidden_features[0], k, activation=activation, dropout=dropout))
        for i in range(len(hidden_features) - 1):
            self.layers.append(
                TAGConv(hidden_features[i], hidden_features[i + 1], k, activation=activation, dropout=dropout))
        self.layers.append(TAGConv(hidden_features[-1], out_features, k))
        self.reset_parameters()

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, x, adj, dropout=0):
        for i in range(len(self.layers)):
            x = self.layers[i](x, adj, dropout=dropout)

        return x

In [63]:
model_tagcn = TAGCN(in_features=num_features, 
                    out_features=num_classes, 
                    hidden_features=[64, 64], 
                    k=2,
                    activation=F.leaky_relu)
model_tagcn.load_state_dict(torch.load("./saved_models/model_tagcn.pt"))
model_tagcn.to(device)
print(model_tagcn)

TAGCN(
  (layers): ModuleList(
    (0): TAGConv(
      (linear): Linear(in_features=906, out_features=64, bias=True)
    )
    (1): TAGConv(
      (linear): Linear(in_features=192, out_features=64, bias=True)
    )
    (2): TAGConv(
      (linear): Linear(in_features=192, out_features=7, bias=True)
    )
  )
)


In [82]:
acc = eval_model(model_tagcn, features, adj, test_mask)
print("Test accuracy (Original): {:.4f}".format(acc))

Test accuracy (Original): 0.8660


In [83]:
acc = eval_model(model_tagcn, torch.cat([features, features_attack]), adj_attack, test_mask)
print("Test accuracy (Attacked): {:.4f}".format(acc))

Test accuracy (Attacked): 0.5540
